In [45]:
import openai
from openai import OpenAI
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from pathlib import Path
import time

# 概述

从较高层面来看，Assistants API 的典型集成具有以下流程：

- 通过定义其自定义指令并选择模型来在 API 中创建助手。如果有帮助，请启用代码解释器、检索和函数调用等工具。
- 当用户开始对话时创建一个线程。
- 当用户提问时将消息添加到线程中。
- **在线程上运行助手以触发响应。这会自动调用相关工具。**

## 步骤1：创建一个 Assistant 

In [2]:
client = OpenAI()


In [5]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

In [6]:
assistant

Assistant(id='asst_q0EW21pauFzIYEygvm2A9f8H', created_at=1699949647, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-3.5-turbo-1106', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

## 步骤2：创建一个Thread

In [7]:
thread = client.beta.threads.create()
thread

Thread(id='thread_vmMDHoQMRHuLOrseONCJzU9I', created_at=1699949699, metadata={}, object='thread')

## 步骤3：将Message 添加到Thread中

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
message

ThreadMessage(id='msg_p71nSLAMIt8VVqlKLYw7AGlL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699949815, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')

## 步骤4：run 这个 Assistant

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # 覆盖助手的默认系统消息。这对于修改每次运行的行为很有用。
    instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [10]:
run

Run(id='run_lujwIqd57oz8PFsgecIaIUVw', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', cancelled_at=None, completed_at=None, created_at=1699950042, expires_at=1699950642, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I', tools=[ToolAssistantToolsCode(type='code_interpreter')])

## 步骤5：打印出 Assistant 的回复

In [11]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run

Run(id='run_lujwIqd57oz8PFsgecIaIUVw', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', cancelled_at=None, completed_at=1699950049, created_at=1699950042, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1699950042, status='completed', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [12]:
message = client.beta.threads.messages.list(
    thread_id=thread.id
)

message

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_3Ro8gba3z2PUjQo1rLer9QE2', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=1699950049, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_lujwIqd57oz8PFsgecIaIUVw', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'), ThreadMessage(id='msg_p71nSLAMIt8VVqlKLYw7AGlL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699949815, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')], object='list', first_id='msg_3Ro8gba3z2PUjQo1rLer9QE2', last_id='msg_p71nSLAMIt8VVqlKLYw7AGlL', has_more=False)

In [13]:
list(message)

[ThreadMessage(id='msg_3Ro8gba3z2PUjQo1rLer9QE2', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=1699950049, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_lujwIqd57oz8PFsgecIaIUVw', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'),
 ThreadMessage(id='msg_p71nSLAMIt8VVqlKLYw7AGlL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699949815, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')]

## 自行测试，步骤6，下一个问题

In [14]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `5x + 20 = 30`. Can you help me?"
)
message

ThreadMessage(id='msg_y58Xvo9dRz00CZA7WgTrm5z0', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `5x + 20 = 30`. Can you help me?'), type='text')], created_at=1699950420, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')

In [15]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # 覆盖助手的默认系统消息。这对于修改每次运行的行为很有用。
    # instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [16]:
message = client.beta.threads.messages.list(
    thread_id=thread.id
)

list(message)

[ThreadMessage(id='msg_tO97JV3ZREoSD4h0RdRHTnwL', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 5x + 20 = 30 is x = 2.'), type='text')], created_at=1699950454, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_QsjWBVpy2J3tvEs4ND4jnn6d', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'),
 ThreadMessage(id='msg_y58Xvo9dRz00CZA7WgTrm5z0', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `5x + 20 = 30`. Can you help me?'), type='text')], created_at=1699950420, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'),
 ThreadMessage(id='msg_3Ro8gba3z2PUjQo1rLer9QE2', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=16

# Assistants 工作原理

- Assistants 可以使用特定指令调用 OpenAI 的模型，以调整其个性和能力。
- Assistants 可以并行访问多个工具。这些可以是 OpenAI 托管的工具（例如代码解释器和知识检索），也可以是您构建/托管的工具（通过函数调用）。
- Assistants 可以访问持久线程。线程通过存储消息历史记录并在对话对于模型上下文长度来说太长时截断它来简化 AI 应用程序开发。您创建一个线程一次，然后只需在用户回复时将消息附加到其中即可。
- Assistants 可以访问多种格式的文件 - 作为其创建的一部分或作为助理和用户之间的线程的一部分。使用工具时，助理还可以创建文件（例如图像、电子表格等）并引用他们在创建的消息中引用的文件。

有个疑问：Assistants 和 Agents 的区别又是什么？

这几个模块的概念：

- Assistant：一个 “专用 AI”，可以使用 OpenAI 模型和调用工具的
- Thread：Assistant 和用户之间的对话会话。线程存储消息并自动处理截断以使内容适合模型的上下文。（cc：这里有没有解决了上下文干扰问题？）
- Message：由Assistant 或用户创建的消息。消息可以包括文本、图像和其他文件。消息以列表形式存储在线程上。
- Run：在线程上调用助手。助手使用它的配置和线程的消息通过调用模型和工具来执行任务。作为运行的一部分，助手将消息附加到线程。
- Run Step：助理在运行过程中所采取的步骤的详细列表。助手可以在运行期间调用工具或创建消息。检查运行步骤可以让您反思助手如何获得最终结果。

## 创建 Assistant

In [39]:
# 数据分析案例

file = client.files.create(
    file=open("data/covid_worldwide.csv", "rb"),
    purpose="assistants"
)

file

FileObject(id='file-uipdQnK2DMrixJISs9qLvoWe', bytes=15552, created_at=1700042664, filename='covid_worldwide.csv', object='file', purpose='assistants', status='processed', status_details=None)

In [40]:
assistant = client.beta.assistants.create(
  name="Data visualizer",
  description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "code_interpreter"}],
  # 每个Assistant最多可以附加20个文件，每个文件的最大大小为512 MB。此外，您的组织上传的所有文件的大小不应超过100GB。
  # file_ids=[file.id]
)

assistant

Assistant(id='asst_sesyhkKoIcOJCYUQttmcqH6U', created_at=1700042666, description='You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.', file_ids=[], instructions=None, metadata={}, model='gpt-3.5-turbo-1106', name='Data visualizer', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

## 管理Threads 和 Messages

线程和消息代表助理和用户之间的对话会话。

线程中可以存储的消息数量没有限制。一旦消息的大小超过模型的上下文窗口，线程将尝试包含尽可能多的适合上下文窗口的消息并删除最旧的消息。

请注意，这种截断策略可能会随着时间的推移而演变。

cc：看来会有一定的识别能力。

In [41]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Create 3 data visualizations based on the trends in this file.",
            "file_ids": [file.id]
        }
    ]
)

thread

Thread(id='thread_EXfeOEv5UkYjFjM69rdbVDvn', created_at=1700042671, metadata={}, object='thread')

In [22]:

thread_message = client.beta.threads.messages.create(
  "thread_RbKLaKF1XnvU1izUJlMLaDGW",
  role="user",
  content="Create 3 data visualizations based on the trends in this file",
  
)
print(thread_message)

ThreadMessage(id='msg_OVpr1NLbpDaxHqT27fwPxuCN', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Create 3 data visualizations based on the trends in this file'), type='text')], created_at=1699963225, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_RbKLaKF1XnvU1izUJlMLaDGW')


In [23]:
list(thread_message)

[('id', 'msg_OVpr1NLbpDaxHqT27fwPxuCN'),
 ('assistant_id', None),
 ('content',
  [MessageContentText(text=Text(annotations=[], value='Create 3 data visualizations based on the trends in this file'), type='text')]),
 ('created_at', 1699963225),
 ('file_ids', []),
 ('metadata', {}),
 ('object', 'thread.message'),
 ('role', 'user'),
 ('run_id', None),
 ('thread_id', 'thread_RbKLaKF1XnvU1izUJlMLaDGW')]

In [33]:
message = client.beta.threads.messages.retrieve(
    thread_id="thread_RbKLaKF1XnvU1izUJlMLaDGW",
    message_id="msg_OVpr1NLbpDaxHqT27fwPxuCN"
)

message_content = message.content[0].text
# cc：关于 annotations 的概念暂时不能理解。
annotations = message_content.annotations
citations = []



In [27]:
for index, annotation in enumerate(annotations):
    print(index)
    # annotations 为空了，没有再继续

## Runs and Run Steps

In [43]:

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  model="gpt-3.5-turbo-1106",
  instructions="additional instructions",
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}]
)